In [9]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import bar_chart_race as bcr
import warnings
%matplotlib inline

In [2]:
scotland_by_health_board_url = "https://www.gov.scot/binaries/content/documents/govscot/publications/statistics/2020/04/coronavirus-covid-19-trends-in-daily-data/documents/covid-19-data-by-nhs-board/covid-19-data-by-nhs-board/govscot%3Adocument/COVID-19%2Bdaily%2Bdata%2B-%2Bby%2BNHS%2BBoard%2B-%2B6%2BSeptember%2B2020.xlsx"
#publicly available url of daily updated covid numbers by health board area.

In [3]:
def get_xlsx_dict(url, xlsx_name):
    resp = requests.get(url)
    output = open(xlsx_name, 'wb') 
    output.write(resp.content)
    output.close()
    dict = pd.read_excel(xlsx_name, sheet_name=None)
    return dict
#takes the url and extracts the xlsx from it and returns the pandas dictionary of sheets

In [4]:
dict_bhb = get_xlsx_dict(scotland_by_health_board_url, '/Users/Ben/Documents/Coding/GitHub/ScotlandBarRace/covid_bhb.xlsx')
#Dictionary of covid numbers by health board

In [5]:
def covid_fmt(df):
    df.drop([0], axis=0).reset_index(drop = True)
    row_start = 0
    for x in range(3):
        if df.iloc[x,1] == 'NHS Ayrshire & Arran':
            df.columns = df.loc[x]
            row_start = x+1
    df = df.iloc[row_start:,:16]
    #Columns Headers are moved from a row to actual columns.
    #the columns are then limited to exclude both the problem row and also 2 columns full of nans
    df = df.replace('*', '0')
    df.head()
    #'*' denotes near zero so reasonable replacement of 0 is used.
    df = df.astype({x:float for x in df.columns[1:]})
    # converts all numbers except date column into float
    df = df.set_index(df.iloc[:,0])
    # sets date columns as index as required for bcr format
    df = df.drop(df.columns[0], axis = 1)
    # drops now reduntant date column
    return df

#Function that formats the selected sheet into suitable format use in BarChartRace

In [6]:
df_tot_cases = dict_bhb['Table 1 - Cumulative cases']
df_icu_cases = dict_bhb['Table 2 - ICU patients']
df_hosp_cases = dict_bhb['Table 3 - Hospital patients']
#dataframe for first three sheets of covid numbers spreadsheet

In [22]:
df = covid_fmt(df_hosp_cases)

In [18]:
#bcr.bar_chart_race(df = df,filename ='') 
# Saves bar race to file path
#for .mp4 make sure ffmpeg is installed
#for .gif make sure ImageMagick is installed

In [21]:
warnings.filterwarnings('ignore')
bcr.bar_chart_race(df = df)
#Simple BarChartRace, as default this is showcasing hospital numbers